In [ ]:
import os
import gc
import logging
import time

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.metrics import mean_squared_error, roc_auc_score, roc_curve, accuracy_score, auc
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn import svm
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.base import clone
from sklearn.pipeline import Pipeline

from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, BatchNormalization, Input, Conv2D
from keras.callbacks import ModelCheckpoint
from keras import metrics
from keras.optimizers import Adam
from keras import backend as K
import keras
from keras.models import Model, Sequential
from keras.models import model_from_json
from keras import regularizers
from keras.losses import binary_crossentropy
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf

In [ ]:
%matplotlib inline

In [ ]:
start_time = time.time()

In [ ]:
train_df = pd.read_csv('../input/train.csv')

In [ ]:
test_df = pd.read_csv('../input/test.csv')

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

In [ ]:
min_max_scaler = MinMaxScaler()

In [ ]:
standart_scaler = StandardScaler()

In [ ]:
robust_scaler = RobustScaler()

In [ ]:
#scaled_train_values = min_max_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])
scaled_train_values = standart_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])
#scaled_train_values = robust_scaler.fit_transform(train_df[train_df.columns.drop(['ID_code', 'target'])])

In [ ]:
train_values, holdout_test_values, train_target_values, holdout_test_target_values = train_test_split(
    #scaled_train_values,
    train_df[train_df.columns.drop(['ID_code', 'target'])].values,
    target_values,
    test_size=0.2,
    random_state=0
)

In [ ]:
print(train_values.shape)
print(train_target_values.shape)
print(holdout_test_values.shape)
print(holdout_test_target_values.shape)

In [ ]:
#test_df.head()

In [ ]:
ID_code = test_df['ID_code'].values

In [ ]:
print(ID_code[:10])

In [ ]:
#tf.py_func

In [ ]:
batch_size = 100

In [ ]:
def auc(y_true, y_pred):
#def auc(y_pred, y_true):
    #print(y_true[:5])
    #print(y_pred[:5])  
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)
    #return tf.py_func(roc_auc_score, (y_true, y_pred), tf.int8)

In [ ]:
checkpoint = ModelCheckpoint(
    'sequential_nn_model_{epoch:02d}.h5',
    monitor='val_acc',
    #monitor=auc,
    verbose=1,
    save_best_only=False,
    #mode='min',
    mode='max'
    #save_weights_only = True
)
reduceLROnPlat = ReduceLROnPlateau(
    monitor='val_loss',
    #monitor=auc,
    factor=0.5,
    patience=7,
    verbose=1,
    mode='min',
    #mode='max',
    epsilon=0.001
)
early = EarlyStopping(
    monitor="val_loss",
    #monitor=auc,
    mode="min",
    #mode='max',
    patience=12
)

In [ ]:
#callbacks_list = [checkpoint, reduceLROnPlat, early]
#callbacks_list = [checkpoint, early]

Провести mean_shift кластеризацию на множестве переменных (features)?

In [ ]:
#submission_predicts = sequential_nn_model.predict(test_df[test_df.columns.drop('ID_code')].values)

In [ ]:
#submission_predicts.shape

In [ ]:
#print(submission_predicts[:5])
#submission_predicts_values = submission_predicts[:, 0]

In [ ]:
#submission_predicts_values.shape

In [ ]:
#np.all(submission_predicts_values == 0)

In [ ]:
#submission_df = pd.DataFrame({'ID_code': ID_code, 'target': submission_predicts_values.astype('float32')})

In [ ]:
#np.any(submission_predicts_values == 1)

In [ ]:
#np.all(submission_predicts_values > 0)

In [ ]:
#np.all(submission_predicts_values < 0)

In [ ]:
#np.any(submission_predicts_values > 0)

In [ ]:
#np.any(submission_predicts_values < 0)

In [ ]:
#submission_predicts_values[submission_predicts_values > 0].shape

In [ ]:
#submission_predicts_values.max()

In [ ]:
#submission_predicts_values.min()

In [ ]:
#submission_predicts_values[submission_predicts_values < 0.245].shape

In [ ]:
#submission_predicts_values[submission_predicts_values >= 0.245].shape

In [ ]:
#submission_predicts_values[submission_predicts_values < 0.245] = 0
#submission_predicts_values[submission_predicts_values >= 0.245] = 1

In [ ]:
#submission_predicts_values[submission_predicts_values == 1].shape

In [ ]:
#submission_df = pd.DataFrame({'ID_code': ID_code, 'target': submission_predicts_values.astype('float32')})

In [ ]:
#submission_df.to_csv('submission_mlp_1.csv', index=False)

In [ ]:
def detect_threshold(classes_ratio, step_size, predicted_values):
    threshold = 0
    values_below_threshold = (predicted_values[predicted_values < threshold]).shape[0]
    values_above_threshold = (predicted_values[predicted_values >= threshold]).shape[0]
    threshold_classes_ratio =  values_abowe_threshold / values_below_threshold
    
    while(threshold_classes_ratio < clasess_ratio):
        threshold += step_size        
        values_below_threshold = (predicted_values[predicted_values < threshold]).shape[0]
        values_above_threshold = (predicted_values[predicted_values >= threshold]).shape[0]
        threshold_classes_ratio =  values_abowe_threshold / values_below_threshold
    predicted_values[predicted_values < threshold] = 0
    predicted_values[predicted_values >= threshold] = 1
    return threshold, predicted_values

In [ ]:
sequential_nn_model = None
#del sequential_nn_model
if sequential_nn_model:
    del sequential_nn_model
sequential_nn_model = Sequential()
#sequential_nn_model.add(Dense(batch_size, input_dim=200, kernel_initializer='normal', activation='sigmoid'))
sequential_nn_model.add(Dense(batch_size, input_shape=(100, 200), kernel_initializer='normal', activation='sigmoid'))
sequential_nn_model.add(Dropout(0.2))
sequential_nn_model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))

In [ ]:
sequential_nn_model.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy', auc])

In [ ]:
start_time = time.time()
sequential_nn_model.fit(train_values, train_target_values, batch_size=100, epochs=20, verbose=1, validation_split=0.2)
print("Run time {} min".format((time.time() - start_time) / 60))

In [ ]:
loss_and_metrics = sequential_nn_model.evaluate(holdout_test_values, holdout_test_target_values, batch_size=100)

In [ ]:
loss_and_metrics